In [1]:
# create Road distance rasters

In [10]:
from shapely.geometry import box
hagfiles = glob("R:/ProjectData/PAG2019/EPCExtent_30cm/Elevation_80cmNPS/HAG_2015/*.tif")
boxes = []
for file in hagfiles:
    with rio.open(file) as src:
        boxes.append(box(*src.bounds))

hagdf = gpd.GeoDataFrame({"File":hagfiles, "geometry":boxes}, geometry=boxes, crs="epsg:2868")
hagdf.to_file(hag_tindex_loc, driver="GPKG")

In [24]:
%load_ext chime
import rasterio as rio
from rasterio.mask import mask
import geopandas as gpd
from shapely.ops import substring
from time import sleep
import numpy as np
from scipy.ndimage import distance_transform_edt
import os
from glob import glob
from datetime import datetime
from shapely.geometry import box
import chime
from joblib import Parallel, delayed
from rasterio.mask import raster_geometry_mask
from math import ceil, floor
import nbimporter
#from KeyFunctions import *
import gdal
from utils import finished

roadDistance_dir = "R:/ProjectData/PAG2019/EPCExtent_30cm/RoadDistances"
os.makedirs(roadDistance_dir, exist_ok=True)
#ortho30cmvrt_loc = os.path.abspath("../EPCExtent_30cm/Orthos/EPC_30cmOrtho_2019.vrt")

#tileindex = gpd.read_file("../EPCExtent_30cm/Orthos/2019_LULC_EPCExtent_30cm_index.shp")
#tifsfootprint = tileindex.geometry.unary_union

asphaultPoly_loc = "../vectors/asphaultPoly.gpkg"
osm_paved = "../vectors/OSMRoadsPimaPaved_20201124.gpkg"
print(f"Reading in {asphaultPoly_loc}...")
road_targets = gpd.read_file(osm_paved)
bad_roads = ["path", "residential", "cycleway", "service","footway","pedestrian","living_street","steps","unknown","bridleway", "unclassified"]


tileIndex_loc = "../vectors/Ortho_5kSubIndex.gpkg"
ruralRegions_loc = r"../vectors/DesertBarrenVegetation_2015.gpkg"
hag_tindex_loc = "R:/ProjectData/PAG2019/EPCExtent_30cm/Elevation_80cmNPS/HAG_2015/HAG_tindex.gpkg"

tileIndex = gpd.read_file(tileIndex_loc)
#tileIndex.filepath = tileIndex.filepath.apply(lambda fp: fp.replace("/mnt/Bertha/PAG2019/EPCExtent_30cm/Orthos_Segmented", "../EPCExtent_30cm/Orthos_Segmentedv2"))
tileIndex["geometry"] = tileIndex["geometry"].apply(lambda g: box(*g.buffer(-50).bounds))

hag_tindex = gpd.read_file(hag_tindex_loc)
rural_regions = gpd.read_file(ruralRegions_loc)

The chime extension is already loaded. To reload it, use:
  %reload_ext chime
Reading in ../vectors/asphaultPoly.gpkg...


C:\Users\BenJames\anaconda3\envs\geospatial\lib\site-packages\geopandas\geodataframe.py:294: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for f in features_lst:
C:\Users\BenJames\anaconda3\envs\geospatial\lib\site-packages\geopandas\geodataframe.py:294: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for f in features_lst:


In [115]:
def done():
    chime.theme("zelda")
    %chime
    chime.theme("material")


def getCommonLandcover(street, writeRasters=False):
    index = street.name
    
    if index % 5000 == 0:
        print(f"On {index} @ {datetime.now()}")
    
    length = street.geometry.length
    substrings = []
    if length > splitlength:
        for i in range(0, ceil(length), splitlength):
            start = i
            end = i + splitlength if i+splitlength < length else length
            substrings.append(substring(street.geometry, start, end))
    else:
        substrings.append(street.geometry)
        
    try:
        commonCovers = []

        for i, string in enumerate(substrings):
            stringPoly = string.buffer(5)

            with rio.open(landcover) as src:
                kwargs = src.profile
                segMask, segTransform, segWindow = raster_geometry_mask(src, [stringPoly], all_touched=True, crop=True, pad=True)
                landcoverData = src.read(1, window=segWindow)
                landcoverData[segMask] = src.nodata
                maskedData = landcoverData[~segMask]
                validValues = maskedData[maskedData!=src.nodata] # for segments that run outside of extent of landcover there may be nodata value on the segement
                unique, frequency = np.unique(validValues, return_counts = True)
                mostCommon = unique[np.argwhere(frequency.max() == frequency)][0][0]
                
                commonCovers.append(coverLookup[mostCommon])
            if writeRasters:
                os.makedirs("./temp", exist_ok=True)
                oras = os.path.join("./temp", str(street.PC_UID) + f"_{i}.tif")
                kwargs.update(transform=segTransform, driver="GTIFF", width=landcoverData.shape[1], height=landcoverData.shape[0])
                with rio.open(oras, 'w', **kwargs) as dst:
                    dst.write(landcoverData, 1)

        return ",".join(set(commonCovers))
    except ValueError:
        #print(f"Index {index} out of coverage")
        return None     


def createDistanceRaster(row, odir, overwrite=False, buffer_size=10):
        #try:
        ofile = os.path.join(odir, f"{row.path}_{row.row}_RoadDist.tif")
        if os.path.exists(ofile) and not overwrite:
            return ofile
        
        geom = row.geometry.buffer(buffer_size)
        
        if "Multi" not in geom.type:
            geom = [geom]
            
        bnds = tileIndex[(tileIndex.path == row.path) & (tileIndex.row == row.row)].geometry.values[0].bounds
        with rio.open(r"R:/ProjectData/PAG2019/EPCExtent_30cm/Orthos/EPC_30cmOrtho_2019.vrt") as src:
            win = rio.windows.from_bounds(*bnds, src.transform).round_offsets().round_lengths().round_shape()
            win_transform = rio.windows.transform(win, src.transform)
            kwargs = src.profile
            kwargs.update(driver="GTIFF", transform=win_transform, width=win.width, height=win.height)
            reso = src.res[0]
            
            roads_burn = rio.features.rasterize(geom, default_value=1, transform=win_transform, out_shape=(win.height, win.width), all_touched=True)
        #with rio.open(rasterpath) as src:
        #    kwargs = src.profile
        #    reso = src.res[0]
        #    ds, ds_trans = mask(src, geom, all_touched=False, crop=False, filled=False)
        
        #distance = distance_transform_edt(ds.mask[0]) * reso
        distance = distance_transform_edt(roads_burn!=1) * reso

        kwargs.update(count=1,dtype=np.uint16, compress="LZW")


        with rio.open(ofile, "w", **kwargs) as dst:
            dst.write(distance.astype(np.uint16), 1)

        print(f"Finished {ofile} - {datetime.now()}")
        return ofile
        #except:
        #print(f"FAILED FOR {os.path.basename(ofile)}")
        #return None

In [123]:
road_targets_over.shape,road_targets_diss.shape

((77173, 38), (4846, 38))

In [124]:
road_targets_over = gpd.overlay(road_targets, tileIndex, how="union")

road_targets_diss = road_targets_over.dissolve(by=["path", "row"], as_index=False)

In [125]:
road_targets_over.shape, road_targets_diss.shape

((72327, 38), (3238, 38))

In [128]:
%%time
road_targets_over = gpd.overlay(road_targets, tileIndex, how="union")

road_targets_diss = road_targets_over.dissolve(by=["path", "row"], as_index=False)

#buffer by 10 so that anything 10 and under is 0 distance
roadFiles = Parallel(n_jobs=6, verbose=5)(delayed(createDistanceRaster)(row, roadDistance_dir, overwrite=True, buffer_size=10) for i, row in road_targets_diss.iterrows())
roadFiles = [f for f in roadFiles if f is not None]
finished()

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    3.7s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:   31.7s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:  1.3min
[Parallel(n_jobs=6)]: Done 276 tasks      | elapsed:  2.3min
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:  3.6min
[Parallel(n_jobs=6)]: Done 636 tasks      | elapsed:  5.2min
[Parallel(n_jobs=6)]: Done 870 tasks      | elapsed:  7.0min
[Parallel(n_jobs=6)]: Done 1140 tasks      | elapsed:  9.1min
[Parallel(n_jobs=6)]: Done 1446 tasks      | elapsed: 11.5min
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed: 14.2min
[Parallel(n_jobs=6)]: Done 2166 tasks      | elapsed: 17.1min
[Parallel(n_jobs=6)]: Done 2580 tasks      | elapsed: 20.3min
[Parallel(n_jobs=6)]: Done 3030 tasks      | elapsed: 24.0min
[Parallel(n_jobs=6)]: Done 3238 out of 3238 | elapsed: 25.8min finished


Wall time: 26min 25s
